# G-Nomix

# Analysis

We will use [snakemake](https://snakemake.readthedocs.io/en/stable/index.html) as workflow manager to parallelize and automatize our analisys. File to run the analysis is `Snakefile_gnomix`. The file `snk_cmmd.sh` is a bash script that allows to run the analysis in a cluster. It makes use of a slurm profile, but it can be easily modified to use other [cluster systems](https://snakemake.readthedocs.io/en/stable/executing/cli.html), or even run sequentially by removinf the `profile` flag.

The workflow has two man steps:

## Local Ancestry Inference
In this step we will use [G-Nomix](https://github.com/AI-sandbox/gnomix), a tool to infer local ancestry.
We will use the following command:

```python3 gnomix {input_vcf} {output_dir} {chromosome} False {pretrained_model}```

Where the input VCF file comes from the previous step, the output directory is the directory where the output files will be saved, the chromosome is the number of the chromosome being analyzed (we focused only on autosomal chromosomes), and the genetic model pretrained by the authors of the tool. The parameter set as `False` indicates the intent of using predicted ancestry for phasing correction and it is recommended if there is evidence of phasing errors in the data.

# Analysis of the results


